In [1]:
import pandas as pd
import numpy as np
import scipy
import librosa

In [11]:
# lese db_snapshot ein
df = pd.read_csv("data/db_data/db_snapshot.csv")
df.head()

,Unnamed: 0,timestamp,duration,node,audio_path,begin_time,end_time,species_name_de,confidence,min_conf,current_location,active_status
0,0,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:10.500000,0 days 00:00:13.500000,Dorngrasmücke,0.911344,0.85,0101000020E6100000000000A0F7A81F40000000C0FA2A...,False
1,1,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:12,0 days 00:00:15,Dorngrasmücke,0.907262,0.85,0101000020E6100000000000A0F7A81F40000000C0FA2A...,False
2,2,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:04.500000,0 days 00:00:07.500000,Dorngrasmücke,0.920276,0.85,0101000020E6100000000000A0F7A81F40000000C0FA2A...,False
3,3,2022-02-23 10:16:33.368821,0 days 00:00:14.097067,3221692,3221692_1645611393-368821.flac,0 days 00:00:01.500000,0 days 00:00:04.500000,Dorngrasmücke,0.976985,0.85,0101000020E6100000000000A0B2A81F4000000060F72A...,True
4,4,2022-02-23 10:16:33.368821,0 days 00:00:14.097067,3221692,3221692_1645611393-368821.flac,0 days 00:00:00,0 days 00:00:03,Dorngrasmücke,0.984349,0.85,0101000020E6100000000000A0B2A81F4000000060F72A...,True


In [12]:
# remove unneeded columns
df.drop(['Unnamed: 0', 'current_location', 'active_status', 'confidence', 'min_conf'], axis=1, inplace=True)
print(df.shape)
df.head()

(129, 7)


,timestamp,duration,node,audio_path,begin_time,end_time,species_name_de
0,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:10.500000,0 days 00:00:13.500000,Dorngrasmücke
1,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:12,0 days 00:00:15,Dorngrasmücke
2,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:04.500000,0 days 00:00:07.500000,Dorngrasmücke
3,2022-02-23 10:16:33.368821,0 days 00:00:14.097067,3221692,3221692_1645611393-368821.flac,0 days 00:00:01.500000,0 days 00:00:04.500000,Dorngrasmücke
4,2022-02-23 10:16:33.368821,0 days 00:00:14.097067,3221692,3221692_1645611393-368821.flac,0 days 00:00:00,0 days 00:00:03,Dorngrasmücke


In [13]:
# filter out duplicates
df = df.drop_duplicates()
print(df.shape)
df.head()

(128, 7)


,timestamp,duration,node,audio_path,begin_time,end_time,species_name_de
0,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:10.500000,0 days 00:00:13.500000,Dorngrasmücke
1,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:12,0 days 00:00:15,Dorngrasmücke
2,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:04.500000,0 days 00:00:07.500000,Dorngrasmücke
3,2022-02-23 10:16:33.368821,0 days 00:00:14.097067,3221692,3221692_1645611393-368821.flac,0 days 00:00:01.500000,0 days 00:00:04.500000,Dorngrasmücke
4,2022-02-23 10:16:33.368821,0 days 00:00:14.097067,3221692,3221692_1645611393-368821.flac,0 days 00:00:00,0 days 00:00:03,Dorngrasmücke


In [20]:
# create dataframes based on species as we do not compare different species, only within the species

df_dorngras = df[df['species_name_de'].str.contains('Dorngrasmücke')]
df_trauer = df[df['species_name_de'].str.contains('Trauerschnäpper')]
df_grau = df[df['species_name_de'].str.contains('Grauschnäpper')]
df_gold = df[df['species_name_de'].str.contains('Goldammer')]
df_blut = df[df['species_name_de'].str.contains('Bluthänfling')]


In [22]:
print(f"Dorngrasmücke: {df_dorngras.shape}")
print(f"Trauerschnäpper: {df_trauer.shape}")
print(f"Grauschnäpper: {df_grau.shape}")
print(f"Goldammer: {df_gold.shape}")
print(f"Bluthänfling: {df_blut.shape}")

Dorngrasmücke: (14, 7)
Trauerschnäpper: (19, 7)
Grauschnäpper: (77, 7)
Goldammer: (6, 7)
Bluthänfling: (12, 7)


In [23]:
df_dorngras.head(20)
# problem: noted timestamp of playback does not match any timestamp in dataframe

,timestamp,duration,node,audio_path,begin_time,end_time,species_name_de
0,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:10.500000,0 days 00:00:13.500000,Dorngrasmücke
1,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:12,0 days 00:00:15,Dorngrasmücke
2,2022-02-23 10:19:30.140225,0 days 00:00:14.028800,6030536,6030536_1645611570-140225.flac,0 days 00:00:04.500000,0 days 00:00:07.500000,Dorngrasmücke
3,2022-02-23 10:16:33.368821,0 days 00:00:14.097067,3221692,3221692_1645611393-368821.flac,0 days 00:00:01.500000,0 days 00:00:04.500000,Dorngrasmücke
4,2022-02-23 10:16:33.368821,0 days 00:00:14.097067,3221692,3221692_1645611393-368821.flac,0 days 00:00:00,0 days 00:00:03,Dorngrasmücke
5,2022-02-23 10:16:30.000177,0 days 00:00:14.165333,6030536,6030536_1645611390-177.flac,0 days 00:00:04.500000,0 days 00:00:07.500000,Dorngrasmücke
6,2022-02-23 10:16:30.000177,0 days 00:00:14.165333,6030536,6030536_1645611390-177.flac,0 days 00:00:03,0 days 00:00:06,Dorngrasmücke
7,2022-02-23 10:16:30.000177,0 days 00:00:14.165333,6030536,6030536_1645611390-177.flac,0 days 00:00:00,0 days 00:00:03,Dorngrasmücke
8,2022-02-23 10:16:14.984882,0 days 00:00:14.165333,6030536,6030536_1645611374-984882.flac,0 days 00:00:04.500000,0 days 00:00:07.500000,Dorngrasmücke
9,2022-02-23 10:16:14.984882,0 days 00:00:14.165333,6030536,6030536_1645611374-984882.flac,0 days 00:00:10.500000,0 days 00:00:13.500000,Dorngrasmücke


In [24]:
df_trauer.head(20)


,timestamp,duration,node,audio_path,begin_time,end_time,species_name_de
13,2022-02-23 10:12:04.393265,0 days 00:00:13.960533,3220764,3220764_1645611124-393265.flac,0 days 00:00:06,0 days 00:00:09,Trauerschnäpper
14,2022-02-23 10:12:04.393265,0 days 00:00:13.960533,3220764,3220764_1645611124-393265.flac,0 days 00:00:03,0 days 00:00:06,Trauerschnäpper
15,2022-02-23 10:12:04.393265,0 days 00:00:13.960533,3220764,3220764_1645611124-393265.flac,0 days 00:00:04.500000,0 days 00:00:07.500000,Trauerschnäpper
16,2022-02-23 10:11:47.158217,0 days 00:00:13.994667,13429496,13429496_1645611107-158217.flac,0 days 00:00:00,0 days 00:00:03,Trauerschnäpper
17,2022-02-23 10:11:47.158217,0 days 00:00:13.994667,13429496,13429496_1645611107-158217.flac,0 days 00:00:03,0 days 00:00:06,Trauerschnäpper
18,2022-02-23 10:11:29.790718,0 days 00:00:14.062933,6030536,6030536_1645611089-790718.flac,0 days 00:00:00,0 days 00:00:03,Trauerschnäpper
19,2022-02-23 10:11:28.871241,0 days 00:00:14.131200,6031380,6031380_1645611088-871241.flac,0 days 00:00:01.500000,0 days 00:00:04.500000,Trauerschnäpper
20,2022-02-23 10:11:28.871241,0 days 00:00:14.131200,6031380,6031380_1645611088-871241.flac,0 days 00:00:00,0 days 00:00:03,Trauerschnäpper
21,2022-02-23 10:11:18.557196,0 days 00:00:13.994667,3221664,3221664_1645611078-557196.flac,0 days 00:00:07.500000,0 days 00:00:10.500000,Trauerschnäpper
22,2022-02-23 10:11:18.557196,0 days 00:00:13.994667,3221664,3221664_1645611078-557196.flac,0 days 00:00:12,0 days 00:00:15,Trauerschnäpper


In [39]:
# put same call detected by multiple boards in one group
df_trauer_1_1 = df.loc[[118, 119]]  # 938song or 279song, probably the latter,  2 nodes
df_trauer_2_1 = df.loc[24:29]       # probably 938song,                         2 nodes
df_trauer_2_2 = df.loc[18:23]       # probably 279song,                         3 nodes
df_trauer_2_3 = df.loc[[16, 17]]    # probably 892call,                         only one node
df_trauer_2_4 = df.loc[13:15]       # probably 383call,                         only one node

# TODO 11:11 differentiate between song and call, as they are temporally close




In [40]:
df_trauer_2_4.head()

,timestamp,duration,node,audio_path,begin_time,end_time,species_name_de
13,2022-02-23 10:12:04.393265,0 days 00:00:13.960533,3220764,3220764_1645611124-393265.flac,0 days 00:00:06,0 days 00:00:09,Trauerschnäpper
14,2022-02-23 10:12:04.393265,0 days 00:00:13.960533,3220764,3220764_1645611124-393265.flac,0 days 00:00:03,0 days 00:00:06,Trauerschnäpper
15,2022-02-23 10:12:04.393265,0 days 00:00:13.960533,3220764,3220764_1645611124-393265.flac,0 days 00:00:04.500000,0 days 00:00:07.500000,Trauerschnäpper
